**Catalog Matching**  
@Author: Ray  
@Time: 2022.07.11  
@Cite: [官方文档](https://docs.astropy.org/en/stable/coordinates/matchsep.html) 

In [135]:
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import match_coordinates_sky

import pandas as pd
import numpy as np

# ^ 禁用同一单元格内的输出覆盖
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 两点之间的距离  

角距离

In [136]:
c1 = SkyCoord('5h23m34.5s', '-69d45m22s', frame='icrs')
c2 = SkyCoord('0h52m44.8s', '-72d49m43s', frame='fk5')  
# 注意到两个源所用的坐标系统不一致，计算时会自动转换
sep = c1.separation(c2)
sep

<Angle 20.74611448 deg>

In [137]:
# 将结果进行单位转换
sep.hour  # * 用小时表示
sep.arcminute  # * 用角分表示
sep.arcsecond  # * 用角秒表示

1.383074298402932

1244.7668685626386

74686.01211375832

线距离

In [138]:
c1 = SkyCoord('5h23m34.5s', '-69d45m22s', distance=70*u.kpc, frame='icrs')
c2 = SkyCoord('0h52m44.8s', '-72d49m43s', distance=80*u.kpc, frame='icrs')
sep = c1.separation_3d(c2)
sep 

<Distance 28.74398816 kpc>

# Matching Catalogs

## 创建样例数据

In [139]:
# 创建样例
# * df_left
ra = [215.22238159,
 215.30378723,
 215.27708435,
 215.27868652,
 215.3032074]

dec = [53.00418472,
 53.05344391,
 53.03545761,
 53.03592682,
 53.05295563]
df_left = pd.DataFrame({'ra': ra, 'dec': dec})
print("="*20 + ' df_left ' + "="*20)
df_left

# * df_right
ra = [215.27868652,
 215.3032074,
 215.26611328,
 215.22912598,
 215.2406311,
 215.27807617,
 215.24153137,
 215.26417542,
 215.29908752,
 215.30310059]

dec = [53.03592682,
 53.05295563,
 53.02712631,
 53.00312042,
 53.00996017,
 53.03676224,
 53.010746,
 53.02722168,
 53.05102539,
 53.05429459]

df_right = pd.DataFrame({'ra': ra, 'dec': dec})
print("="*20 + ' df_right ' + "="*20)
df_right

==================== df_left ====================


,ra,dec
0,215.222382,53.004185
1,215.303787,53.053444
2,215.277084,53.035458
3,215.278687,53.035927
4,215.303207,53.052956


==================== df_right ====================


,ra,dec
0,215.278687,53.035927
1,215.303207,53.052956
2,215.266113,53.027126
3,215.229126,53.003120
4,215.240631,53.009960
5,215.278076,53.036762
6,215.241531,53.010746
7,215.264175,53.027222
8,215.299088,53.051025
9,215.303101,53.054295


## Matching

In [140]:
# 传入两个catalog的坐标列
ra1 = df_left['ra']
dec1 = df_left['dec']
ra2 = df_right['ra']
dec2 = df_right['dec']
c1 = SkyCoord(ra=ra1*u.degree, dec=dec1*u.degree)
c2 = SkyCoord(ra=ra2*u.degree, dec=dec2*u.degree)

matching的方法1: `c1.match_to_catalog_sky(c2)`

In [141]:
# 遍历c1中的每个源，给出c2中距离该源最近源在c2中的索引，角距离，和距离
idx, d2d, d3d = c1.match_to_catalog_sky(c2)
# ! 注意顺序，在c2中找c1中的源的最近邻，结果的长度跟c1一致
idx
d2d
d3d

array([3, 1, 0, 0, 1])

<Angle [0.00419576, 0.0005999 , 0.0010716 , 0.        , 0.        ] deg>

<Quantity [7.32298541e-05, 1.04702846e-05, 1.87029115e-05, 0.00000000e+00,
           0.00000000e+00]>

matching的方法2: `match_coordinates_sky(c1, c2)`  

In [142]:
idx, d2d, d3d = match_coordinates_sky(c1, c2)
# ! 注意顺序，在c2中找c1中的源的最近邻，结果的长度跟c1一致
idx
d2d
d3d

array([3, 1, 0, 0, 1])

<Angle [0.00419576, 0.0005999 , 0.0010716 , 0.        , 0.        ] deg>

<Quantity [7.32298541e-05, 1.04702846e-05, 1.87029115e-05, 0.00000000e+00,
           0.00000000e+00]>

设置阈值&输出结果

In [143]:
max_sep = 1.0*u.arcsec
sep_constraint = d2d < max_sep
sep_constraint # 返回的是与c1长度相同的bool

array([False, False, False,  True,  True])

In [144]:
df_right
idx
idx[sep_constraint]

,ra,dec
0,215.278687,53.035927
1,215.303207,53.052956
2,215.266113,53.027126
3,215.229126,53.003120
4,215.240631,53.009960
5,215.278076,53.036762
6,215.241531,53.010746
7,215.264175,53.027222
8,215.299088,53.051025
9,215.303101,53.054295


array([3, 1, 0, 0, 1])

array([0, 1])

In [145]:
# df_left中匹配到的源
df_left_matches = df_left[sep_constraint]
df_left_matches

# df_right中匹配到的源
df_right_matches = df_right.iloc[idx[sep_constraint]]
df_right_matches

,ra,dec
3,215.278687,53.035927
4,215.303207,53.052956


,ra,dec
0,215.278687,53.035927
1,215.303207,53.052956


# Matching Module  
第一个cell是探索版  
第二个cell是实用版

In [146]:
# 设置match的阈值
max_sep = 1 * u.arcsec 

df1 = df_left.copy()
df2 = df_right.copy()

# 输入坐标列
ra1 = df1['ra']
dec1 = df1['dec']

ra2 = df2['ra']
dec2 = df2['dec']

# matching
c1 = SkyCoord(ra=ra1*u.degree, dec=dec1*u.degree)
c2 = SkyCoord(ra=ra2*u.degree, dec=dec2*u.degree)
idx, d2d, d3d = c1.match_to_catalog_sky(c2)
sep_constraint = d2d < max_sep

# 向c1添加match信息
df1['idx'] = idx
df1['sep_constraint'] = sep_constraint
df1['d2d'] = d2d.to('arcsec').value  # 单位: 角秒
df1

# 设置c2的索引列为id，用于merge
df2['id'] = df2.index
df2

# 合并两表
df_merge = pd.merge(left=df1, right=df2, left_on="idx", right_on="id")
df_merge

# 筛出符合sep_constraint的源
df_out = df_merge.query("sep_constraint==True")
df_out = df_out.drop(columns=['idx', 'id', 'd2d', 'sep_constraint'])
df_out

,ra,dec,idx,sep_constraint,d2d
0,215.222382,53.004185,3,False,15.104742
1,215.303787,53.053444,1,False,2.159651
2,215.277084,53.035458,0,False,3.857752
3,215.278687,53.035927,0,True,0.000000
4,215.303207,53.052956,1,True,0.000000


,ra,dec,id
0,215.278687,53.035927,0
1,215.303207,53.052956,1
2,215.266113,53.027126,2
3,215.229126,53.003120,3
4,215.240631,53.009960,4
5,215.278076,53.036762,5
6,215.241531,53.010746,6
7,215.264175,53.027222,7
8,215.299088,53.051025,8
9,215.303101,53.054295,9


,ra_x,dec_x,idx,sep_constraint,d2d,ra_y,dec_y,id
0,215.222382,53.004185,3,False,15.104742,215.229126,53.003120,3
1,215.303787,53.053444,1,False,2.159651,215.303207,53.052956,1
2,215.303207,53.052956,1,True,0.000000,215.303207,53.052956,1
3,215.277084,53.035458,0,False,3.857752,215.278687,53.035927,0
4,215.278687,53.035927,0,True,0.000000,215.278687,53.035927,0


,ra_x,dec_x,ra_y,dec_y
2,215.303207,53.052956,215.303207,53.052956
4,215.278687,53.035927,215.278687,53.035927


In [148]:
#
# ^ ********** matching module **********

# 设置match的阈值
max_sep = 1 * u.arcsec 

df1 = df_left.copy()
df2 = df_right.copy()

# 输入坐标列（不加list容易出错）
ra1 = list(df1['ra'])
dec1 = list(df1['dec'])

ra2 = list(df2['ra'])
dec2 = list(df2['dec'])

# matching
c1 = SkyCoord(ra=ra1*u.degree, dec=dec1*u.degree)
c2 = SkyCoord(ra=ra2*u.degree, dec=dec2*u.degree)
idx, d2d, d3d = c1.match_to_catalog_sky(c2)
sep_constraint = d2d < max_sep

# 向c1添加match信息
df1['idx'] = idx
df1['sep_constraint'] = sep_constraint
df1['d2d'] = d2d.to('arcsec').value  # 单位: 角秒

# 设置c2的索引列为id，用于merge
df2['id'] = df2.index

# 合并两表
df_merge = pd.merge(left=df1, right=df2, left_on="idx", right_on="id")

# 筛出符合sep_constraint的源
df_out = df_merge.query("sep_constraint==True")
df_out = df_out.drop(columns=['idx', 'id', 'd2d', 'sep_constraint'])
df_out.reset_index(inplace=True, drop=True)
df_out

,ra_x,dec_x,ra_y,dec_y
0,215.303207,53.052956,215.303207,53.052956
1,215.278687,53.035927,215.278687,53.035927
